In [2]:
import numpy as np
from pathlib import Path
import json
import io
import polars as pl
import floodlight
from floodlight.io.secondspectrum import read_position_data_jsonl
from floodlight.io.secondspectrum import read_teamsheets_from_meta_json
import json
import tempfile



In [3]:
filepath_position = Path("SecondSpectrum/Second Spectrum/Anderlecht - Club Brugge/tracking-produced.jsonl")

In [4]:
filepath_metadata = Path("SecondSpectrum/Second Spectrum/Anderlecht - Club Brugge/rsc-bru_metadata.json")

In [5]:
metadata = pl.read_json(filepath_metadata)

In [6]:
# Load raw metadata
with open(filepath_metadata) as f:
    raw_meta = json.load(f)

# Extract the nested `data` block
meta_flat = raw_meta["data"]
meta_flat["fps"] = 50

for period in meta_flat.get("periods", []):
    period["startFrameIdx"] = period.pop("startFrameClock", None)
    period["endFrameIdx"] = period.pop("endFrameClock", None)

# Save it to a temporary cleaned file
with open("cleaned_metadata.json", "w") as f:
    json.dump(meta_flat, f)

In [7]:
print(meta_flat.keys()) 

dict_keys(['startTime', 'year', 'month', 'day', 'description', 'id', 'externalIds', 'homeTeam', 'awayTeam', 'venue', 'pitchLength', 'pitchWidth', 'homeScore', 'awayScore', 'periods', 'fps'])


In [8]:
meta_flat

{'startTime': 1736703000000,
 'year': 2025,
 'month': 1,
 'day': 12,
 'description': 'RSC - BRU : 2025-1-12',
 'id': 'd074dbef-20ce-4602-aa1e-cd7a6acc641a',
 'externalIds': {},
 'homeTeam': {'id': '147449ab-9af9-4edc-bbcd-e37529b6ec6b',
  'externalIds': {'optaId': '241', 'fspId': '50074'},
  'players': [{'name': 'Mads Kikkenborg',
    'number': '16',
    'position': 'SUB',
    'id': '65d4435d-8017-457f-bbc7-cb1f17157591',
    'externalIds': {'optaId': '466310',
     'performId': '191f96bw3vkg1nt8545rvbw4q',
     'fspId': '250113521'}},
   {'name': 'Amuzu',
    'number': '7',
    'position': 'MF',
    'id': 'ae963650-6c2f-4b50-b8b2-20ebd9b573cc',
    'externalIds': {'optaId': '437741',
     'performId': '5don85s7097g5df42m429b8ix',
     'fspId': '250087449'}},
   {'name': 'T. Leoni',
    'number': '17',
    'position': 'MF',
    'id': '9605505f-4535-43b7-9afc-b89a8845b913',
    'externalIds': {'fspId': '250100836'}},
   {'name': 'M. Rits',
    'number': '23',
    'position': 'MF',
    '

In [9]:
meta_flat_path = Path("cleaned_metadata1.json")

In [10]:
with open(meta_flat_path, 'r') as f:
    metadata = json.load(f)
    print(metadata["fps"])

50



 "periods" data uses "startFrameClock" and "endFrameClock" rather than "startFrameIdx" and "endFrameIdx
 homePlayers & awayPlayers toegevoegd, heel die home & away team lijst weggehaald

In [11]:
read_position_data_jsonl(
    filepath_position,
    meta_flat_path
    #teamsheet_home=None, 
    #teamsheet_away=None
)

IndexError: index -1736703123000 is out of bounds for axis 0 with size 3054601

In [69]:
metadata

ok,requestId,data
bool,str,struct[15]
true,"""b487cb4e-ea0a-4684-9408-cc67cf…","{1736703000000,2025,1,12,""RSC - BRU : 2025-1-12"",""d074dbef-20ce-4602-aa1e-cd7a6acc641a"",{},{""147449ab-9af9-4edc-bbcd-e37529b6ec6b"",{""241"",""50074""},[{""Mads Kikkenborg"",""16"",""SUB"",""65d4435d-8017-457f-bbc7-cb1f17157591"",{""250113521"",""466310"",""191f96bw3vkg1nt8545rvbw4q""}}, {""Amuzu"",""7"",""MF"",""ae963650-6c2f-4b50-b8b2-20ebd9b573cc"",{""250087449"",""437741"",""5don85s7097g5df42m429b8ix""}}, … {""Ali Maamar"",""79"",""MF"",""a43daa37-dfcc-480c-be54-5e1716cd3043"",{""250202834"",null,null}}]},{""c1ee4c03-dc60-435b-96c5-56f9db553827"",{""413"",""50043""},[{""S. Mignolet"",""22"",""GK"",""777bd523-6f9b-4671-b79e-87a77ec62da5"",{""66797"",""byfrgpc6tgz8lto1gor03y5sl"",""98078""}}, {""Ordoñez"",""4"",""DF"",""2ee50c94-78ed-452c-943b-bef72cfb80b1"",{null,null,null}}, … {""Raphael Onyedika"",""15"",""MF"",""c6975643-0cee-423f-92a9-a407a1282779"",{""503548"",""mkq18lpl3d44wxq974mci5wq"",""250135101""}}]},{""951d377a-03fd-402d-aa0f-b0d81f6f4741"",{""63395""}},103.458978,66.97553,0,3,[{1,1736703123000,1736706177600,true}, {2,1736707281320,1736710183120,false}]}"


In [73]:
unnested_metadata = metadata.unnest("data")

In [ ]:
unnested_metadata_json = unnested_metadata.write_json()

In [92]:
unnested_metadata_json

'[{"ok":true,"requestId":"b487cb4e-ea0a-4684-9408-cc67cf549700","startTime":1736703000000,"year":2025,"month":1,"day":12,"description":"RSC - BRU : 2025-1-12","id":"d074dbef-20ce-4602-aa1e-cd7a6acc641a","externalIds":{},"homeTeam":{"id":"147449ab-9af9-4edc-bbcd-e37529b6ec6b","externalIds":{"optaId":"241","fspId":"50074"},"players":[{"name":"Mads Kikkenborg","number":"16","position":"SUB","id":"65d4435d-8017-457f-bbc7-cb1f17157591","externalIds":{"fspId":"250113521","optaId":"466310","performId":"191f96bw3vkg1nt8545rvbw4q"}},{"name":"Amuzu","number":"7","position":"MF","id":"ae963650-6c2f-4b50-b8b2-20ebd9b573cc","externalIds":{"fspId":"250087449","optaId":"437741","performId":"5don85s7097g5df42m429b8ix"}},{"name":"T. Leoni","number":"17","position":"MF","id":"9605505f-4535-43b7-9afc-b89a8845b913","externalIds":{"fspId":"250100836","optaId":null,"performId":null}},{"name":"M. Rits","number":"23","position":"MF","id":"025ac194-dcf5-4659-bbee-7cf9b159b094","externalIds":{"fspId":"250012557

In [91]:
with tempfile.NamedTemporaryFile(mode='w+', suffix='.json', delete=False) as tmpfile:
    json.dump(unnested_metadata_json, tmpfile)
    tmpfile_path = tmpfile.name
    
read_position_data_jsonl(
    filepath_position,
    tmpfile_path)

AttributeError: 'str' object has no attribute 'get'

In [58]:
metadata['data']

data
struct[15]
"{1736703000000,2025,1,12,""RSC - BRU : 2025-1-12"",""d074dbef-20ce-4602-aa1e-cd7a6acc641a"",{},{""147449ab-9af9-4edc-bbcd-e37529b6ec6b"",{""241"",""50074""},[{""Mads Kikkenborg"",""16"",""SUB"",""65d4435d-8017-457f-bbc7-cb1f17157591"",{""466310"",""191f96bw3vkg1nt8545rvbw4q"",""250113521""}}, {""Amuzu"",""7"",""MF"",""ae963650-6c2f-4b50-b8b2-20ebd9b573cc"",{""437741"",""5don85s7097g5df42m429b8ix"",""250087449""}}, … {""Ali Maamar"",""79"",""MF"",""a43daa37-dfcc-480c-be54-5e1716cd3043"",{null,null,""250202834""}}]},{""c1ee4c03-dc60-435b-96c5-56f9db553827"",{""413"",""50043""},[{""S. Mignolet"",""22"",""GK"",""777bd523-6f9b-4671-b79e-87a77ec62da5"",{""66797"",""byfrgpc6tgz8lto1gor03y5sl"",""98078""}}, {""Ordoñez"",""4"",""DF"",""2ee50c94-78ed-452c-943b-bef72cfb80b1"",{null,null,null}}, … {""Raphael Onyedika"",""15"",""MF"",""c6975643-0cee-423f-92a9-a407a1282779"",{""503548"",""mkq18lpl3d44wxq974mci5wq"",""250135101""}}]},{""951d377a-03fd-402d-aa0f-b0d81f6f4741"",{""63395""}},103.458978,66.97553,0,3,[{1,1736703123000,1736706177600,true}, {2,1736707281320,1736710183120,false}]}"


In [61]:
# test floodlight.io.secondspectrum
event_data = read_teamsheets_from_meta_json(path_to_metadata)


KeyError: 'homePlayers'

In [ ]:
dataset = pl.read_ndjson(path_to_jsonl)

In [18]:
dataset

period,frameIdx,gameClock,wallClock,homePlayers,awayPlayers,ball,live,lastTouch
i64,i64,f64,i64,list[struct[4]],list[struct[4]],struct[2],bool,str
1,0,0.0,1736703123000,"[{""025ac194-dcf5-4659-bbee-7cf9b159b094"",23,[-12.86, -10.79, 0.0],0.0}, {""04aace5a-de9f-44fa-9be6-7cc45a32a4af"",54,[-19.56, -25.26, 0.0],0.0}, … {""e1eadfe8-a29b-4869-bac7-fe10709884b5"",4,[-23.0, -14.83, 0.0],0.0}]","[{""2ee50c94-78ed-452c-943b-bef72cfb80b1"",4,[8.87, 5.31, 0.0],0.0}, {""3fdd2d58-c370-4fbb-ae12-d9845b4120e3"",68,[0.09, 9.1, 0.0],0.0}, … {""df80d30e-807b-4d6b-b7c4-5aced8ed5f9f"",8,[0.16, -15.82, 0.0],0.0}]","{[-0.21, 0.07, 0.48],18.7}",false,"""home"""
1,1,0.04,1736703123040,"[{""025ac194-dcf5-4659-bbee-7cf9b159b094"",23,[-12.85, -10.81, 0.0],0.19}, {""04aace5a-de9f-44fa-9be6-7cc45a32a4af"",54,[-19.56, -25.25, 0.0],0.13}, … {""e1eadfe8-a29b-4869-bac7-fe10709884b5"",4,[-23.02, -14.83, 0.0],0.19}]","[{""2ee50c94-78ed-452c-943b-bef72cfb80b1"",4,[8.88, 5.31, 0.0],0.07}, {""3fdd2d58-c370-4fbb-ae12-d9845b4120e3"",68,[0.09, 9.09, 0.0],0.09}, … {""df80d30e-807b-4d6b-b7c4-5aced8ed5f9f"",8,[0.15, -15.82, 0.0],0.25}]","{[0.5, -0.08, 0.38],18.62}",false,"""home"""
1,2,0.08,1736703123080,"[{""025ac194-dcf5-4659-bbee-7cf9b159b094"",23,[-12.84, -10.82, 0.0],0.38}, {""04aace5a-de9f-44fa-9be6-7cc45a32a4af"",54,[-19.56, -25.24, 0.0],0.26}, … {""e1eadfe8-a29b-4869-bac7-fe10709884b5"",4,[-23.03, -14.84, 0.0],0.38}]","[{""2ee50c94-78ed-452c-943b-bef72cfb80b1"",4,[8.88, 5.3, 0.0],0.13}, {""3fdd2d58-c370-4fbb-ae12-d9845b4120e3"",68,[0.09, 9.09, 0.0],0.17}, … {""df80d30e-807b-4d6b-b7c4-5aced8ed5f9f"",8,[0.13, -15.82, 0.0],0.5}]","{[1.26, -0.19, 0.48],18.55}",false,"""home"""
1,3,0.12,1736703123120,"[{""025ac194-dcf5-4659-bbee-7cf9b159b094"",23,[-12.85, -10.84, 0.0],0.38}, {""04aace5a-de9f-44fa-9be6-7cc45a32a4af"",54,[-19.57, -25.23, 0.0],0.2}, … {""e1eadfe8-a29b-4869-bac7-fe10709884b5"",4,[-23.05, -14.84, 0.0],0.38}]","[{""2ee50c94-78ed-452c-943b-bef72cfb80b1"",4,[8.88, 5.3, 0.0],0.17}, {""3fdd2d58-c370-4fbb-ae12-d9845b4120e3"",68,[0.09, 9.08, 0.0],0.15}, … {""df80d30e-807b-4d6b-b7c4-5aced8ed5f9f"",8,[0.11, -15.82, 0.0],0.78}]","{[2.02, -0.29, 0.53],18.48}",true,"""away"""
1,4,0.16,1736703123160,"[{""025ac194-dcf5-4659-bbee-7cf9b159b094"",23,[-12.85, -10.85, 0.0],0.38}, {""04aace5a-de9f-44fa-9be6-7cc45a32a4af"",54,[-19.57, -25.22, 0.0],0.19}, … {""e1eadfe8-a29b-4869-bac7-fe10709884b5"",4,[-23.06, -14.84, 0.0],0.43}]","[{""2ee50c94-78ed-452c-943b-bef72cfb80b1"",4,[8.88, 5.29, 0.0],0.21}, {""3fdd2d58-c370-4fbb-ae12-d9845b4120e3"",68,[0.09, 9.07, 0.0],0.18}, … {""df80d30e-807b-4d6b-b7c4-5aced8ed5f9f"",8,[0.08, -15.82, 0.0],1.12}]","{[2.78, -0.39, 0.59],18.4}",true,"""away"""
…,…,…,…,…,…,…,…,…
2,148907,2901.64,1736710182960,"[{""04aace5a-de9f-44fa-9be6-7cc45a32a4af"",54,[32.01, 13.7, 0.0],1.97}, {""6f6faff9-64e1-4500-9fb5-70210df86ad2"",32,[25.88, -2.59, 0.0],1.9}, … {""f82683e5-362d-40eb-8d66-043587ac72a8"",20,[3.4, -0.27, 0.0],1.93}]","[{""181465f6-a5c1-48ce-98b4-54185bbdd4a4"",58,[-5.23, -11.25, 0.0],2.18}, {""3c5a517a-ca83-4b72-b0fe-61a7ab13902d"",14,[0.66, 5.64, 0.0],4.06}, … {""c904ad13-a205-47d1-aeec-e54ae830cd16"",55,[46.14, -16.37, 0.0],1.36}]","{null,0.0}",false,"""home"""
2,148908,2901.68,1736710183000,"[{""04aace5a-de9f-44fa-9be6-7cc45a32a4af"",54,[31.94, 13.7, 0.0],1.75}, {""6f6faff9-64e1-4500-9fb5-70210df86ad2"",32,[25.81, -2.62, 0.0],1.79}, … {""f82683e5-362d-40eb-8d66-043587ac72a8"",20,[3.33, -0.28, 0.0],1.75}]","[{""181465f6-a5c1-48ce-98b4-54185bbdd4a4"",58,[-5.28, -11.32, 0.0],1.98}, {""3c5a517a-ca83-4b72-b0fe-61a7ab13902d"",14,[0.51, 5.61, 0.0],3.73}, … {""c904ad13-a205-47d1-aeec-e54ae830cd16"",55,[46.09, -16.38, 0.0],1.26}]","{null,0.0}",false,"""home"""
2,148909,2901.72,1736710183040,"[{""04aace5a-de9f-44fa-9be6-7cc45a32a4af"",54,[31.87, 13.7, 0.0],1.63}, {""6f6faff9-64e1-4500-9fb5-70210df86ad2"",32,[25.75, -2.65, 0.0],1.66}, … {""f82683e5-362d-40eb-8d66-043587ac72a8"",20,[3.26, -0.28, 0.0],1.63}]","[{""181465f6-a5c1-48ce-98b4-54185bbdd4a4"",58,[

In [15]:
home, _, _ = dataset.get()

AttributeError: 'DataFrame' object has no attribute 'get'